In [231]:
import requests

from bs4 import BeautifulSoup

import time


from mplsoccer import Pitch, VerticalPitch, FontManager, Sbopen
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsbombpy import sb

In [232]:
#Scraping data for the Premier League teams.

premier_league_data = []

In [233]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

clubs_of_interest = ['Liverpool', 'Manchester City', 'Arsenal','Manchester United','Tottenham Hotspur','Chelsea']


data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
standings_table = soup.select('table.stats_table')[0]

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")
    
    if team_name in clubs_of_interest:
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        passing = pd.read_html(data.text, match='Passing')[0]
        passing.columns = passing.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT','PK', 'PKatt']], on='Date').merge(passing[['Date', 'Cmp', 'Att','Cmp%']],on='Date').merge(misc[['Date', 'CrdY', 'CrdR','Fls','Fld']], on='Date')

        except ValueError:

            continue

        team_data['Team'] = team_name
        premier_league_data.append(team_data)
        time.sleep(3)

        


C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\1091890056.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\1091890056.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\1091890056.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match='Passing')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\1091890056.py:37: FutureWarning: Passing literal html to 'r

In [234]:
premier_league = pd.concat(premier_league_data)

In [235]:
premier_league.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
       'Referee', 'Match Report', 'Notes', 'Sh', 'SoT', 'PK', 'PKatt', 'Cmp',
       'Cmp', 'Cmp', 'Cmp', 'Att', 'Att', 'Att', 'Att', 'Cmp%', 'Cmp%', 'Cmp%',
       'Cmp%', 'CrdY', 'CrdR', 'Fls', 'Fld', 'Team'],
      dtype='object')

In [236]:
premier_league = premier_league.loc[:, ~premier_league.columns.duplicated()]

In [237]:
premier_league = premier_league[premier_league['Comp'] == 'Premier League']


In [239]:
premier_league = premier_league[['Date','Round','Venue', 'Result', 'GF', 'GA','Opponent','xG', 'xGA','Poss','Sh','SoT','PK','PKatt',
       'Cmp', 'Att', 'Cmp%','CrdY','CrdR','Fls','Fld','Team']]

In [100]:
""" 
team_names = premier_league['Opponent'].unique()

# Define country codes to remove
country_codes = ['nl','fr', 'es', 'pt', 'rs','ch', 'dk', 'de', 'tr']

# Remove country codes from team names
cleaned_team_names = [name.split(maxsplit=1)[-1].strip() if name.split(maxsplit=1)[0] in country_codes else name for name in team_names]

name_mapping = dict(zip(team_names, cleaned_team_names))

premier_league['Opponent'] = premier_league['Opponent'].replace(name_mapping)

"""


In [240]:
premier_league

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
1,2023-08-12,Matchweek 1,Home,W,2,1,Nott'ham Forest,0.8,1.2,78.0,...,0,0,716.0,807.0,88.7,2,0,12,12,Arsenal
2,2023-08-21,Matchweek 2,Away,W,1,0,Crystal Palace,2.0,1.0,53.0,...,1,1,436.0,527.0,82.7,3,1,10,13,Arsenal
3,2023-08-26,Matchweek 3,Home,D,2,2,Fulham,3.2,0.6,71.0,...,1,1,644.0,741.0,86.9,0,0,6,5,Arsenal
4,2023-09-03,Matchweek 4,Home,W,3,1,Manchester Utd,2.3,0.9,55.0,...,0,0,490.0,583.0,84.0,2,0,8,7,Arsenal
5,2023-09-17,Matchweek 5,Away,W,1,0,Everton,1.0,0.3,74.0,...,0,0,627.0,721.0,87.0,1,0,10,12,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-02-04,Matchweek 23,Home,L,2,4,Wolves,1.7,2.0,64.0,...,0,0,597.0,693.0,86.1,3,0,15,11,Chelsea
32,2024-02-12,Matchweek 24,Away,W,3,1,Crystal Palace,1.4,0.6,77.0,...,0,0,784.0,897.0,87.4,2,0,7,14,Chelsea
33,2024-02-17,Matchweek 25,Away,D,1,1,Manchester City,1.4,2.6,30.0,...,0,0,266.0,336.0,79.2,3,0,12,7,Chelsea
36,2024-03-02,Matchweek 27,Away,D,2,2,Brentford,1.8,1.8,67.0,...,0,0,580.0,695.0,83.5,1,0,9,10,Chelsea


In [102]:
# Assuming df is your DataFrame
# Save DataFrame to a CSV file with specified delimiter and encoding
premier_league.to_csv('premier_league.csv', sep=',', encoding='utf-8', index=False)


In [241]:
#Scraping data for La Liga teams.


la_liga_data = []

standings_url = 'https://fbref.com/en/comps/12/La-Liga-Stats'

clubs_of_interest = ['Real Madrid','Barcelona', 'Atletico Madrid']


data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
standings_table = soup.select('table.stats_table')[0]

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")
    
    if team_name in clubs_of_interest:
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        passing = pd.read_html(data.text, match='Passing')[0]
        passing.columns = passing.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT','Dist', 'PK', 'PKatt']], on='Date').merge(passing[['Date', 'Cmp', 'Att','Cmp%']],on='Date').merge(misc[['Date', 'CrdY', 'CrdR','Fls','Fld']], on='Date')

        except ValueError:

            continue

        team_data['Team'] = team_name
        la_liga_data.append(team_data)
        time.sleep(3)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2512612702.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2512612702.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2512612702.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match='Passing')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2512612702.py:42: FutureWarning: Passing literal html to 'r

In [242]:
la_liga = pd.concat(la_liga_data)

In [243]:
la_liga = la_liga.loc[:, ~la_liga.columns.duplicated()]

In [244]:
la_liga = la_liga[la_liga['Comp'] == 'La Liga']

In [245]:
la_liga = la_liga[['Date','Round','Venue', 'Result', 'GF', 'GA','Opponent','xG', 'xGA','Poss','Sh','SoT','PK','PKatt',
       'Cmp', 'Att', 'Cmp%','CrdY','CrdR','Fls','Fld','Team']]

In [246]:
la_liga

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
0,2023-08-12,Matchweek 1,Away,W,2.0,0.0,Athletic Club,0.9,0.4,54.0,...,0,0,556.0,639.0,87.0,1,0,6,16,Real Madrid
1,2023-08-19,Matchweek 2,Away,W,3.0,1.0,Almería,2.0,1.3,57.0,...,0,0,548.0,622.0,88.1,0,0,13,9,Real Madrid
2,2023-08-25,Matchweek 3,Away,W,1.0,0.0,Celta Vigo,1.4,1.2,63.0,...,0,1,589.0,672.0,87.6,3,0,14,19,Real Madrid
3,2023-09-02,Matchweek 4,Home,W,2.0,1.0,Getafe,2.8,0.4,76.0,...,0,0,674.0,782.0,86.2,2,0,12,15,Real Madrid
4,2023-09-17,Matchweek 5,Home,W,2.0,1.0,Real Sociedad,2.0,1.6,52.0,...,0,0,477.0,558.0,85.5,3,0,11,13,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2024-02-17,Matchweek 25,Home,W,5,0,Las Palmas,3.5,0.7,36.0,...,1,1,263.0,365.0,72.1,2,0,19,9,Atletico Madrid
37,2024-02-24,Matchweek 26,Away,D,2,2,Almería,1.5,0.5,60.0,...,0,0,493.0,597.0,82.6,1,0,12,13,Atletico Madrid
39,2024-03-03,Matchweek 27,Home,W,2,1,Betis,3.0,0.6,42.0,...,0,1,372.0,481.0,77.3,1,0,9,12,Atletico Madrid
40,2024-03-09,Matchweek 28,Away,L,0,2,Cádiz,0.9,1.1,60.0,...,0,0,493.0,612.0,80.6,5,0,15,19,Atletico Madrid


In [247]:
#Scraping data for El Calcio teams.


calcio_data = []

standings_url = 'https://fbref.com/en/comps/11/Serie-A-Stats'

clubs_of_interest = ['Internazionale', 'Milan', 'Juventus']


data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
standings_table = soup.select('table.stats_table')[0]

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")
    
    if team_name in clubs_of_interest:
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        passing = pd.read_html(data.text, match='Passing')[0]
        passing.columns = passing.columns.droplevel()


        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT','Dist', 'PK', 'PKatt']], on='Date').merge(passing[['Date', 'Cmp', 'Att','Cmp%']],on='Date').merge(misc[['Date', 'CrdY', 'CrdR','Fls','Fld']], on='Date')

        except ValueError:

            continue

        team_data['Team'] = team_name
        calcio_data.append(team_data)
        time.sleep(3)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2725623093.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2725623093.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2725623093.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match='Passing')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2725623093.py:43: FutureWarning: Passing literal html to 'r

In [248]:
calcio = pd.concat(calcio_data)

calcio = calcio.loc[:, ~calcio.columns.duplicated()]

calcio = calcio[(calcio['Comp'] == 'Serie A')]

calcio = calcio[['Date','Round','Venue', 'Result', 'GF', 'GA','Opponent','xG', 'xGA','Poss','Sh','SoT','PK','PKatt',
       'Cmp', 'Att', 'Cmp%','CrdY','CrdR','Fls','Fld','Team']]

In [249]:
calcio

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
0,2023-08-19,Matchweek 1,Home,W,2,0,Monza,2.7,0.8,47.0,...,0,0,448.0,509.0,88.0,1,0,8,13,Internazionale
1,2023-08-28,Matchweek 2,Away,W,2,0,Cagliari,0.9,0.6,67.0,...,0,0,616.0,730.0,84.4,1,0,10,9,Internazionale
2,2023-09-03,Matchweek 3,Home,W,4,0,Fiorentina,3.7,0.4,40.0,...,1,1,303.0,371.0,81.7,1,0,13,11,Internazionale
3,2023-09-16,Matchweek 4,Home,W,5,1,Milan,2.6,1.1,41.0,...,1,1,305.0,377.0,80.9,2,0,16,12,Internazionale
5,2023-09-24,Matchweek 5,Away,W,1,0,Empoli,1.6,0.3,61.0,...,0,0,517.0,621.0,83.3,2,0,11,14,Internazionale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,2024-02-17,Matchweek 25,Away,D,2.0,2.0,Hellas Verona,1.8,0.6,56.0,...,1,1,416.0,509.0,81.7,0,0,10,7,Juventus
27,2024-02-25,Matchweek 26,Home,W,3.0,2.0,Frosinone,2.0,0.4,62.0,...,0,0,572.0,668.0,85.6,2,0,12,5,Juventus
28,2024-03-03,Matchweek 27,Away,L,1.0,2.0,Napoli,1.6,2.1,30.0,...,0,0,218.0,295.0,73.9,4,0,16,6,Juventus
29,2024-03-10,Matchweek 28,Home,D,2.0,2.0,Atalanta,1.3,0.5,52.0,...,0,0,457.0,556.0,82.2,0,0,12,10,Juventus


In [250]:
#Scraping data for Bundesliga teams.


bundesliga_data = []

standings_url = 'https://fbref.com/en/comps/20/Bundesliga-Stats'

clubs_of_interest = ['Bayern Munich', 'Dortmund']


data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
standings_table = soup.select('table.stats_table')[0]

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")
    
    if team_name in clubs_of_interest:
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        passing = pd.read_html(data.text, match='Passing')[0]
        passing.columns = passing.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT','Dist', 'PK', 'PKatt']], on='Date').merge(passing[['Date', 'Cmp', 'Att','Cmp%']],on='Date').merge(misc[['Date', 'CrdY', 'CrdR','Fls','Fld']], on='Date')

        except ValueError:

            continue

        team_data['Team'] = team_name
        bundesliga_data.append(team_data)
        time.sleep(3)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2894255621.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2894255621.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2894255621.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match='Passing')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2894255621.py:42: FutureWarning: Passing literal html to 'r

In [251]:
bundesliga = pd.concat(bundesliga_data)

bundesliga = bundesliga.loc[:, ~bundesliga.columns.duplicated()]

bundesliga = bundesliga[(bundesliga['Comp'] == 'Bundesliga')]

bundesliga = bundesliga[['Date','Round','Venue', 'Result', 'GF', 'GA','Opponent','xG', 'xGA','Poss','Sh','SoT','PK','PKatt',
       'Cmp', 'Att', 'Cmp%','CrdY','CrdR','Fls','Fld','Team']]

In [252]:
bundesliga

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
1,2023-08-18,Matchweek 1,Away,W,4.0,0.0,Werder Bremen,2.8,0.6,72.0,...,0,0,629.0,707.0,89.0,1,0,11,17,Bayern Munich
2,2023-08-27,Matchweek 2,Home,W,3.0,1.0,Augsburg,2.8,0.7,61.0,...,1,1,562.0,657.0,85.5,1,0,14,6,Bayern Munich
3,2023-09-02,Matchweek 3,Away,W,2.0,1.0,M'Gladbach,2.4,0.3,65.0,...,0,0,598.0,695.0,86.0,2,0,20,5,Bayern Munich
4,2023-09-15,Matchweek 4,Home,D,2.0,2.0,Leverkusen,2.1,2.1,51.0,...,0,0,539.0,634.0,85.0,0,0,6,6,Bayern Munich
6,2023-09-23,Matchweek 5,Home,W,7.0,0.0,Bochum,4.8,0.1,62.0,...,1,1,534.0,616.0,86.7,0,0,8,9,Bayern Munich
8,2023-09-30,Matchweek 6,Away,D,2.0,2.0,RB Leipzig,2.3,0.7,65.0,...,1,1,639.0,727.0,87.9,5,0,11,9,Bayern Munich
10,2023-10-08,Matchweek 7,Home,W,3.0,0.0,Freiburg,2.4,0.1,69.0,...,0,0,725.0,815.0,89.0,1,0,7,8,Bayern Munich
11,2023-10-21,Matchweek 8,Away,W,3.0,1.0,Mainz 05,1.8,1.1,62.0,...,0,0,554.0,646.0,85.8,2,0,10,12,Bayern Munich
13,2023-10-28,Matchweek 9,Home,W,8.0,0.0,Darmstadt 98,5.8,0.7,54.0,...,0,0,472.0,536.0,88.1,1,1,15,13,Bayern Munich
15,2023-11-04,Matchweek 10,Away,W,4.0,0.0,Dortmund,3.2,1.0,51.0,...,0,0,479.0,554.0,86.5,2,0,6,13,Bayern Munich


In [253]:
#Scraping data for Bundesliga teams.


france_data = []

standings_url = 'https://fbref.com/en/comps/13/Ligue-1-Stats'

clubs_of_interest = ['Paris Saint Germain']


data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')
standings_table = soup.select('table.stats_table')[0]

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")
    
    if team_name in clubs_of_interest:
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        passing = pd.read_html(data.text, match='Passing')[0]
        passing.columns = passing.columns.droplevel()

        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/misc/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        misc = pd.read_html(data.text, match='Miscellaneous Stats')[0]
        misc.columns = misc.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT','Dist', 'PK', 'PKatt']], on='Date').merge(passing[['Date', 'Cmp', 'Att','Cmp%']],on='Date').merge(misc[['Date', 'CrdY', 'CrdR','Fls','Fld']], on='Date')

        except ValueError:

            continue

        team_data['Team'] = team_name
        france_data.append(team_data)
        time.sleep(3)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\3274135151.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match='Scores & Fixtures')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\3274135151.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match='Shooting')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\3274135151.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match='Passing')[0]
C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\3274135151.py:42: FutureWarning: Passing literal html to 'r

In [254]:
ligue_1 = pd.concat(france_data)

ligue_1 = ligue_1.loc[:, ~ligue_1.columns.duplicated()]

ligue_1 = ligue_1[(ligue_1['Comp'] == 'Ligue 1')]

ligue_1 = ligue_1[['Date','Round','Venue', 'Result', 'GF', 'GA','Opponent','xG', 'xGA','Poss','Sh','SoT','PK','PKatt',
       'Cmp', 'Att', 'Cmp%','CrdY','CrdR','Fls','Fld','Team']]

In [255]:
ligue_1

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
0,2023-08-12,Matchweek 1,Home,D,0.0,0.0,Lorient,1.2,0.1,78.0,...,0,0,964.0,1051.0,91.7,0,0,8,5,Paris Saint Germain
1,2023-08-19,Matchweek 2,Away,D,1.0,1.0,Toulouse,2.1,1.2,76.0,...,1,1,720.0,802.0,89.8,3,0,11,7,Paris Saint Germain
2,2023-08-26,Matchweek 3,Home,W,3.0,1.0,Lens,1.5,0.5,66.0,...,0,0,695.0,761.0,91.3,0,0,8,16,Paris Saint Germain
3,2023-09-03,Matchweek 4,Away,W,4.0,1.0,Lyon,4.5,2.1,69.0,...,1,1,781.0,843.0,92.6,2,0,5,9,Paris Saint Germain
4,2023-09-15,Matchweek 5,Home,L,2.0,3.0,Nice,1.0,1.3,69.0,...,0,0,640.0,743.0,86.1,2,0,8,17,Paris Saint Germain
6,2023-09-24,Matchweek 6,Home,W,4.0,0.0,Marseille,1.9,0.7,76.0,...,0,0,751.0,845.0,88.9,0,0,11,14,Paris Saint Germain
7,2023-09-30,Matchweek 7,Away,D,0.0,0.0,Clermont Foot,3.0,1.0,67.0,...,0,0,534.0,617.0,86.5,3,0,16,11,Paris Saint Germain
9,2023-10-08,Matchweek 8,Away,W,3.0,1.0,Rennes,3.5,1.1,58.0,...,0,0,586.0,663.0,88.4,0,0,15,12,Paris Saint Germain
10,2023-10-21,Matchweek 9,Home,W,3.0,0.0,Strasbourg,2.0,0.4,76.0,...,1,1,800.0,867.0,92.3,2,0,11,9,Paris Saint Germain
12,2023-10-29,Matchweek 10,Away,W,3.0,2.0,Brest,2.1,1.8,56.0,...,0,1,473.0,546.0,86.6,4,0,10,14,Paris Saint Germain


In [256]:

# Concatenate DataFrames
teams_analysis = pd.concat([premier_league, la_liga, calcio, bundesliga, ligue_1])

# Resetting index
teams_analysis.reset_index(drop=True, inplace=True)


In [257]:
teams_analysis

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,PK,PKatt,Cmp,Att,Cmp%,CrdY,CrdR,Fls,Fld,Team
0,2023-08-12,Matchweek 1,Home,W,2,1,Nott'ham Forest,0.8,1.2,78.0,...,0,0,716.0,807.0,88.7,2,0,12,12,Arsenal
1,2023-08-21,Matchweek 2,Away,W,1,0,Crystal Palace,2.0,1.0,53.0,...,1,1,436.0,527.0,82.7,3,1,10,13,Arsenal
2,2023-08-26,Matchweek 3,Home,D,2,2,Fulham,3.2,0.6,71.0,...,1,1,644.0,741.0,86.9,0,0,6,5,Arsenal
3,2023-09-03,Matchweek 4,Home,W,3,1,Manchester Utd,2.3,0.9,55.0,...,0,0,490.0,583.0,84.0,2,0,8,7,Arsenal
4,2023-09-17,Matchweek 5,Away,W,1,0,Everton,1.0,0.3,74.0,...,0,0,627.0,721.0,87.0,1,0,10,12,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,2024-02-17,Matchweek 22,Away,W,2.0,0.0,Nantes,1.5,1.3,77.0,...,1,1,835.0,921.0,90.7,0,0,12,10,Paris Saint Germain
415,2024-02-25,Matchweek 23,Home,D,1.0,1.0,Rennes,1.9,0.8,66.0,...,1,1,643.0,743.0,86.5,5,0,13,16,Paris Saint Germain
416,2024-03-01,Matchweek 24,Away,D,0.0,0.0,Monaco,1.5,1.6,68.0,...,0,0,573.0,677.0,84.6,0,0,13,8,Paris Saint Germain
417,2024-03-10,Matchweek 25,Home,D,2.0,2.0,Reims,1.2,1.3,64.0,...,0,0,553.0,642.0,86.1,2,0,10,13,Paris Saint Germain


In [258]:
teams_analysis.to_csv('teams_analysis_final.csv')

In [263]:
df_xG= teams_analysis[['Date','Team', 'Opponent','GF','GA', 'xG', 'xGA','Result']]

In [265]:
df_xG['id'] = np.arange(1, len(df_xG) + 1)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15424\2721771380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xG['id'] = np.arange(1, len(df_xG) + 1)


In [269]:
match_ids = list(df_xG['id'])

In [276]:
!pip install prettytable
import pandas as pd
from prettytable import PrettyTable
import numpy as np
import time


In [341]:
num_simulations = 5000
df_xpts = pd.DataFrame(columns=['match_id', 'team', 'xPts'])
n=0
for match_id in match_ids:
    team1 = df_xG[df_xG['id'] == match_id]['Team'].values[0]
    team2 = df_xG[df_xG['id'] == match_id]['Opponent'].values[0]
    #team1 = 'Local'
    #team2 = 'Visitante'
    input_home_team_xg = df_xG.loc[df_xG['id'] == match_id, 'xG'].values[0]
    input_away_team_xg = df_xG.loc[df_xG['id'] == match_id, 'xGA'].values[0]

    #print the simulation table and run simulations
    count_home_wins = 0
    count_home_loss = 0
    count_away_wins = 0
    count_away_loss = 0
    count_draws = 0
    score_mat = []
    tot_sim_time = 0
    #sim_table = PrettyTable(["N de Simulación #", input_home_team, input_away_team, "Loc Ganó", "Vis Ganó", "EMPATE", "DIF DE GOLES"])

    for i in range(num_simulations):
        #get simulation start time
        start_time = time.time()
        #run the sim - generate a random Poisson distribution
        target_home_goals_scored = np.random.poisson(input_home_team_xg)
        target_away_goals_scored = np.random.poisson(input_away_team_xg)
        home_win = 0
        away_win = 0
        draw = 0
        margin = 0
        # if more goals for home team => home team wins
        if target_home_goals_scored > target_away_goals_scored:
            count_home_wins += 1
            count_away_loss += 1
            home_win = 1
            margin = target_home_goals_scored - target_away_goals_scored
        # if more goals for away team => away team wins
        elif target_home_goals_scored < target_away_goals_scored:
            count_away_wins += 1
            count_home_loss += 1
            away_win = 1
            margin = target_away_goals_scored - target_home_goals_scored
        elif target_home_goals_scored == target_away_goals_scored:
            draw = 1
            count_draws += 1
            margin = target_away_goals_scored - target_home_goals_scored
        # add score to score matrix
        score_mat.append((target_home_goals_scored, target_away_goals_scored))
        #get end time
        end_time = time.time()
        #add the time to the total simulation time
        tot_sim_time += round((end_time - start_time),5)
        #add the info to the simulation table
        #sim_table.add_row([i+1, target_home_goals_scored, target_away_goals_scored, home_win, away_win, draw, margin])
    # calculate probabilities to win/lose/draw
    #print(sim_table)

    home_win_probability = round((count_home_wins/num_simulations * 100),2)
    away_win_probability = round((count_away_wins/num_simulations * 100),2)
    draw_probability = round((count_draws/num_simulations * 100),2)

    sim_table_stats = PrettyTable(["Total # de sim", f"{team1} gana", f"{team2} gana", "Empates"])
    sim_table_stats.add_row([num_simulations, count_home_wins, count_away_wins, count_draws])
    sim_table_stats.add_row(["-", str(home_win_probability)+"%", str(away_win_probability)+"%", str(draw_probability)+"%"])
    #print(sim_table_stats)
    #print('-'*50)

    #calculate expected Pts and print a summary
    home_xPts = (home_win_probability / 100) * 3.0 + (draw_probability / 100) * 1.0 + (away_win_probability / 100) * 0.0
    away_xPts = (away_win_probability / 100) * 3.0 + (draw_probability / 100) * 1.0 + (home_win_probability / 100) * 0.0
    #print(team1, "probabilidad de ganar %:", home_win_probability, "|| xPts =", round(home_xPts,2))
    #print(team2, "probabilidad de ganar %:", away_win_probability, "|| xPts =", round(away_xPts,2))
    #print("Probabilidad de empate %:", draw_probability)
    lista = [match_id, team1, home_xPts]
    lista2 = [match_id, team2, away_xPts]
    df_xpts.loc[len(df_xpts)] = lista
    df_xpts.loc[len(df_xpts)] = lista2
    #print(df_xpts)
    #print('-'*50)
    n+=1

In [340]:
df_xpts.head(15)

,team,xPts
0,Arsenal,1.0256
1,Nott'ham Forest,1.6718
2,Arsenal,2.0178
3,Crystal Palace,0.7680
4,Arsenal,2.6950
5,Fulham,0.2080
6,Arsenal,2.2382
7,Manchester Utd,0.5876
8,Arsenal,1.9428
9,Everton,0.6990


In [347]:
df_xpts

,match_id,team,xPts
0,1,Arsenal,1.0350
1,1,Nott'ham Forest,1.6566
2,2,Arsenal,2.0676
3,2,Crystal Palace,0.7236
4,3,Arsenal,2.7100
...,...,...,...
833,417,Monaco,1.4586
834,418,Paris Saint Germain,1.2970
835,418,Reims,1.4338
836,419,Paris Saint Germain,0.9302


In [335]:
df_xpts.drop(columns = 'match_id', inplace = True)

In [329]:
teams =['Real Madrid',
 'Barcelona',
 'Bayern Munich',
 'Manchester United',
 'Juventus',
 'Chelsea',
 'Arsenal',
 'Liverpool',
 'Milan',
 'Internazionale',
 'Dortmund',
 'Paris Saint Germain',
 'Manchester City',
 'Atletico Madrid',
 'Tottenham Hotspur']

In [343]:

filtered_df = df_xpts.drop_duplicates(subset='match_id', keep='first')


In [352]:
len(filtered_df[filtered_df['team'] == 'Arsenal'])

28

In [346]:
filtered_df.to_csv('teams_Xpts.csv')

In [358]:


df_xpts_final.groupby('team').sum().sort_values(by='xPts', ascending=False).head(15).reset_index()

,team,xPts
0,Internazionale,60.1516
1,Bayern Munich,59.4052
2,Barcelona,59.0982
3,Arsenal,58.9376
4,Real Madrid,56.5688
5,Manchester City,55.6008
6,Juventus,54.2106
7,Liverpool,52.8582
8,Atletico Madrid,50.9422
9,Paris Saint Germain,48.2040


In [357]:
df_xpts_final.drop(columns = 'match_id', inplace=True)

In [327]:
list(df_xpts_final['team'])

['Manchester City',
 'Arsenal',
 'Barcelona',
 'Liverpool',
 'Bayern Munich',
 'Real Madrid',
 'Internazionale',
 'Juventus',
 'Chelsea',
 'Atletico Madrid',
 'Paris Saint Germain',
 'Milan',
 'Dortmund',
 'Tottenham Hotspur',
 'Manchester United']